# Implement American Put options

In this notebook, we will simple implement American put options using numpy

In [208]:
import numpy as np

In [261]:
# Deifne class for American put option
class BinomialTreeAmerican:
    """Binomial tree for American put option"""

    def __init__(self, params):
        self.__dict__.update(params)    # load model parameters
        self.disc = 1 + self.r          # Compute discount rate
        self.risk_probs = (self.disc - self.d) / (self.u - self.d)   
        self.N = self.N
        self.delta0 = self.calculate_delta0()
        self.V0 = self.calculate_Vn(0)
        # self.X0 = self.calculate_x0()

    def calculate_Sn(self, n):
        # Compute potential price outcomes
        return (
            self.S0
            * self.d ** (np.arange(n, -1, -1))
            * self.u ** (np.arange(0, n + 1, 1))
        )

    def calculate_VN(self):
        # Compute potential price outcomes
        return np.maximum(self.K - self.calculate_Sn(self.N), 0)
    
    def calculate_Vn(self, n):
        # calculate Vn at time n
        Vn = self.calculate_VN()
        
        for i in np.arange(self.N, n, -1):
            # Compute Vn for each step
            Vn = ((self.risk_probs) * Vn[1 : i + 1] + (1 - self.risk_probs) * Vn[0:i]) / self.disc
            Vn = np.maximum(Vn, self.K - self.calculate_Sn(i-1))

        return Vn
    
    def calculate_delta0(self):
        # Compute risk neutal share holding
        self.d0 = np.diff(self.calculate_Vn(1))[0] / ((self.u - self.d) * self.S0)
        return self.d0
    
    def calculate_deltan(self, n):
        # Calculate delta at time n
        return np.diff(self.calculate_Vn(n + 1)) / np.diff(self.calculate_Sn(n + 1))
    
    def calculate_x0(self):
        # Calculate X0
        return sum(
            (1 / self.disc)
            * self.calculate_vn(1)
            * np.array([1 - self.risk_probs, self.risk_probs])
        )
    
    def calculate_Cn(self, n):
        # Calculate cash flow from time n
        Vnp1 = self.calculate_Vn(n+1)

        return self.calculate_Vn(n) - (self.risk_probs * Vnp1[1:n+2] + (1 - self.risk_probs) * Vnp1[:n+1]) / self.disc
        




In [262]:
# Define parameters
u = 2
d = 1 / u
r = 0.25
N= 3
S0 = 4

# Define risk probabilities
risk_p = (1 + r - d) / (u - d)


params = {
    "u": u,
    "d": d,
    "S0": S0,
    "N": N, 
    "r": r,
    "K": 6,
}

# Create model
model = BinomialTreeAmerican(params=params)

Vnp1 = model.calculate_Vn(2)

In [267]:
n = 2

Vnp1 = model.calculate_Vn(n+1)
Vn = model.calculate_Vn(n)

In [271]:
Vnd = (model.risk_probs * Vnp1[1:n+2] + (1-model.risk_probs) * Vnp1[:n+1])/model.disc

In [272]:
Vn - Vnd

array([1.2, 0.4, 0. ])

In [273]:
model.calculate_Cn(n)

array([1.2, 0.4, 0. ])